In [110]:
import pandas as pd
import requests

params = {
  "format": "json"
    
}

df = pd.DataFrame(requests.get("https://waterwatch.usgs.gov/webservices/realtime", params = params) .json()["sites"])
df["site_no"] = df["site_no"].astype("int")

county_lookup = pd.read_csv("county_lookup.csv").dropna()
county_lookup["county_fips"] = county_lookup["county_fips"].astype("int").astype("str")

prepend_zero = lambda s: "0" + s if len(s) == 4 else s
county_lookup["county_fips"] = county_lookup["county_fips"].apply(prepend_zero)

plot_df = pd.merge(county_lookup,df,on="site_no")
plot_df = plot_df[["state_nm","state_code","county_nm","county_fips","dec_lat_va","dec_long_va","percentile","station_nm","site_no"]]



In [59]:
test = plot_df.groupby(["state_nm","county_nm","county_fips","state_code"])[["percentile"]].median().reset_index()



In [20]:
#test[test["state_nm"].isin(["Alabama"]) & (test["county_nm"].isin(["Baldwin"]))]
# dict(zip(test['county_fips'], test['county_nm']))
test_dict = dict(zip(test['state_code'], test['state_nm']))
test_dict["CA"]


'California'

In [42]:
params = {
  "format": "json",
}
test_7 = pd.DataFrame(requests.get("https://waterwatch.usgs.gov/webservices/flows7d", params = params) .json()["sites"]).dropna()
test_14 = pd.DataFrame(requests.get("https://waterwatch.usgs.gov/webservices/flows14d", params = params) .json()["sites"]).dropna()
test_28 = pd.DataFrame(requests.get("https://waterwatch.usgs.gov/webservices/flows28d", params = params) .json()["sites"]).dropna()

In [46]:
test_7

,site_no,station_nm,dec_lat_va,dec_long_va,huc_cd,flow,flow_dt,flow_unit,flow_nday,class,percentile,percent_median,percent_mean
4,01011500,"St. Francis River near Connors, New Brun",47.2070,-68.9564,01010001,317.14,2023-03-20,cfs,7.0,5,70.20,138.92,106.29
15,01018500,"St. Croix River at Vanceboro, Maine",45.5683,-67.4283,01050001,1615.71,2023-03-20,cfs,7.0,7,94.53,280.44,224.57
16,01019000,"Grand Lake Stream at Grand Lake Stream,",45.1725,-67.7689,01050001,255.29,2023-03-20,cfs,7.0,5,35.85,63.30,51.26
17,01021000,"St. Croix River at Baring, Maine",45.1369,-67.3181,01050001,2488.33,2023-03-20,cfs,6.0,5,35.13,87.90,78.80
23,01027240,S. Br. Penobscot River nr Canada Falls L,45.8739,-69.9950,01020001,121.29,2023-03-20,cfs,7.0,0,19.78,84.65,70.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8883,442114088085701,"GLRI WATERWAY NUMBER 5 NR WRIGHTSTOWN, W",44.3540,-88.1492,04030204,0.02,2023-03-20,cfs,7.0,0,52.50,200.00,66.67
8884,442119088085501,"GLRI WATERWAY NUMBER 4 NR WRIGHTSTOWN, W",44.3553,-88.1487,04030204,0.05,2023-03-20,cfs,7.0,0,75.00,500.00,166.67
8885,442155087330001,DISCOVERY FARMS WATERWAY KD2 NEAR KEWAUN,44.3653,-87.5499,04030101,0.08,2023-03-20,cfs,7.0,0,33.33,20.51,20.51
8886,442405087414401,DISCOVERY FARMS WATERWAY KD1 NEAR KEWAUN,44.4013,-87.6956,04030101,3.81,2023-03-20,cfs,7.0,0,75.00,12700.00,9525.00


In [47]:
test_7 = test_7[test_7["percentile"] != 0]
test_14 = test_14[test_14["percentile"] != 0]
test_28 = test_28[test_28["percentile"] != 0]

In [48]:
test_7 = test_7.rename(columns={'percentile': 'percentile_7'}) 
test_14 = test_14.rename(columns={'percentile': 'percentile_14'}) 
test_28 = test_28.rename(columns={'percentile': 'percentile_28'}) 


In [111]:
from functools import reduce
df_list = []

for n in ["7","14","28"]:
    df = pd.DataFrame(requests.get(f"https://waterwatch.usgs.gov/webservices/flows{n}d", params = params) .json()["sites"]).dropna()
    df = df[df["percentile"] != 0]
    df["site_no"] = df["site_no"].astype("int")
    df = df[["site_no","percentile"]]
    df = df.rename(columns={'percentile': f'{n}'}) 
    df_list.append(df)
    
plot_df = plot_df[plot_df["percentile"] != 0].dropna()
plot_df = plot_df.rename(columns={'percentile': "overall"}) 

df_list.append(plot_df)


test = reduce(lambda left, right: left.merge(right, on='site_no'), df_list)


In [112]:
  test = test.groupby(["state_nm", "county_nm", "county_fips", "state_code" ])[["overall", "7", "14","28"]].median().reset_index()
test

,state_nm,county_nm,county_fips,state_code,overall,7,14,28
0,Alabama,Baldwin County,01003,AL,43.930,42.860,37.670,23.590
1,Alabama,Bibb County,01007,AL,29.610,43.280,34.760,32.130
2,Alabama,Blount County,01009,AL,35.015,50.080,53.290,48.250
3,Alabama,Calhoun County,01015,AL,67.310,41.400,67.230,54.880
4,Alabama,Chambers County,01017,AL,56.570,56.660,68.270,41.330
...,...,...,...,...,...,...,...,...
2143,Wyoming,Park County,56029,WY,33.415,35.835,30.800,28.510
2144,Wyoming,Sweetwater County,56037,WY,50.560,58.180,54.970,48.670
2145,Wyoming,Teton County,56039,WY,20.840,26.195,29.485,30.375
2146,Wyoming,Uinta County,56041,WY,37.060,34.780,33.870,35.000


In [113]:
test.melt(id_vars=['state_nm', 'county_nm','county_fips','state_code'], var_name='n', value_name='percentile')

,state_nm,county_nm,county_fips,state_code,n,percentile
0,Alabama,Baldwin County,01003,AL,overall,43.930
1,Alabama,Bibb County,01007,AL,overall,29.610
2,Alabama,Blount County,01009,AL,overall,35.015
3,Alabama,Calhoun County,01015,AL,overall,67.310
4,Alabama,Chambers County,01017,AL,overall,56.570
...,...,...,...,...,...,...
8587,Wyoming,Park County,56029,WY,28,28.510
8588,Wyoming,Sweetwater County,56037,WY,28,48.670
8589,Wyoming,Teton County,56039,WY,28,30.375
8590,Wyoming,Uinta County,56041,WY,28,35.000
